## Step 1: Modeling 'churn' (account cancellation)

---

In this first step of the project, we will focus on modeling 'churn', which refers to the cancellation of customer accounts. The objective is to develop a predictive model that can identify customers who are likely to churn based on the provided dataset.

Import necessary libraries including numpy, pandas, sklearn, and PyCaret.

In [43]:
import numpy as np
import pandas as pd
import time
from joblib import dump

In [44]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

In [45]:
from pycaret.classification import *

Read feature and outcome data from CSV files. 

- Feature data is read from 
    - '../data/raw/orange_small_train.data' 
- and outcome data is read from 
    - '../data/raw/orange_small_train_churn.labels'.

In [46]:
features = pd.read_csv('../data/raw/orange_small_train.data', sep = '\t', na_filter = True)
outcome = pd.read_csv('../data/raw/orange_small_train_churn.labels', header = None)

Print the shape of feature data, its first few records, as well as the shape and unique values of outcome data.

In [47]:
print(features.shape)
features.head()

(50000, 230)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN


In [48]:
outcome.rename(columns={0: 'Churn'})

,Churn
0,-1
1,1
2,-1
3,-1
4,-1
...,...
49995,-1
49996,-1
49997,-1
49998,-1


In [49]:
print(outcome.shape)
print(np.unique(outcome))
outcome = outcome.rename(columns={0: 'Churn'})
outcome.head()

(50000, 1)
[-1  1]


,Churn
0,-1
1,1
2,-1
3,-1
4,-1


Divide variables into numeric and categorical, assuming the first 190 columns are numeric and the rest are categorical.

In [50]:
all_vars = np.array(features.columns)
num_vars = np.array(all_vars[:190])
num_vars

array(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8',
       'Var9', 'Var10', 'Var11', 'Var12', 'Var13', 'Var14', 'Var15',
       'Var16', 'Var17', 'Var18', 'Var19', 'Var20', 'Var21', 'Var22',
       'Var23', 'Var24', 'Var25', 'Var26', 'Var27', 'Var28', 'Var29',
       'Var30', 'Var31', 'Var32', 'Var33', 'Var34', 'Var35', 'Var36',
       'Var37', 'Var38', 'Var39', 'Var40', 'Var41', 'Var42', 'Var43',
       'Var44', 'Var45', 'Var46', 'Var47', 'Var48', 'Var49', 'Var50',
       'Var51', 'Var52', 'Var53', 'Var54', 'Var55', 'Var56', 'Var57',
       'Var58', 'Var59', 'Var60', 'Var61', 'Var62', 'Var63', 'Var64',
       'Var65', 'Var66', 'Var67', 'Var68', 'Var69', 'Var70', 'Var71',
       'Var72', 'Var73', 'Var74', 'Var75', 'Var76', 'Var77', 'Var78',
       'Var79', 'Var80', 'Var81', 'Var82', 'Var83', 'Var84', 'Var85',
       'Var86', 'Var87', 'Var88', 'Var89', 'Var90', 'Var91', 'Var92',
       'Var93', 'Var94', 'Var95', 'Var96', 'Var97', 'Var98', 'Var99',
       'Var100', 'Va

In [51]:
cat_vars = np.array(all_vars[190:])
cat_vars

array(['Var191', 'Var192', 'Var193', 'Var194', 'Var195', 'Var196',
       'Var197', 'Var198', 'Var199', 'Var200', 'Var201', 'Var202',
       'Var203', 'Var204', 'Var205', 'Var206', 'Var207', 'Var208',
       'Var209', 'Var210', 'Var211', 'Var212', 'Var213', 'Var214',
       'Var215', 'Var216', 'Var217', 'Var218', 'Var219', 'Var220',
       'Var221', 'Var222', 'Var223', 'Var224', 'Var225', 'Var226',
       'Var227', 'Var228', 'Var229', 'Var230'], dtype=object)

Replace empty cells (represented as empty strings) with NaN (Not a Number).

In [52]:
features = features.replace('', np.nan)

Convert numeric columns to float data type and categorical columns to category data type.

In [53]:
for col in num_vars:
    features[col] = features[col].astype('float')
for col in cat_vars:
    features[col] = features[col].astype('category')

Check empty cells in each column and compute the count of empty cells in each column.

In [54]:
features.isna()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,True,True,True,True,True,False,False,True,True,True,...,False,False,False,True,True,False,False,False,True,True
1,True,True,True,True,True,False,False,True,True,True,...,False,False,False,True,True,False,False,False,True,True
2,True,True,True,True,True,False,False,True,True,True,...,False,False,False,True,False,False,False,False,False,True
3,True,True,True,True,True,True,False,True,True,True,...,False,False,False,True,True,False,False,False,True,True
4,True,True,True,True,True,False,False,True,True,True,...,False,False,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,True,True,True,True,True,False,False,True,True,True,...,False,False,False,True,True,False,False,False,True,True
49996,True,True,True,True,True,False,False,True,True,True,...,False,False,False,True,False,False,False,False,False,True
49997,True,True,True,True,True,False,False,True,True,True,...,False,False,False,True,False,False,False,False,True,True
49998,True,True,True,False,True,True,True,True,True,True,...,False,False,False,True,True,False,False,False,True,True


In [55]:
empty_entries_per_column = features.isna().sum(axis = 0)
empty_entries_per_column

Var1      49298
Var2      48759
Var3      48760
Var4      48421
Var5      48513
          ...  
Var226        0
Var227        0
Var228        0
Var229    28432
Var230    50000
Length: 230, dtype: int64

Calculate the maximum number of allowed empty entries based on a 50% threshold and keep only columns that have a count less than or equal to the threshold.

In [56]:
num_entries = len(features)
threshold = 0.75
num_entries * threshold

37500.0

In [57]:
keep_vars = np.array(features.columns[(empty_entries_per_column <= (num_entries * threshold))])
keep_vars

array(['Var6', 'Var7', 'Var13', 'Var21', 'Var22', 'Var24', 'Var25',
       'Var28', 'Var35', 'Var38', 'Var44', 'Var57', 'Var65', 'Var72',
       'Var73', 'Var74', 'Var76', 'Var78', 'Var81', 'Var83', 'Var85',
       'Var94', 'Var109', 'Var112', 'Var113', 'Var119', 'Var123',
       'Var125', 'Var126', 'Var132', 'Var133', 'Var134', 'Var140',
       'Var143', 'Var144', 'Var149', 'Var153', 'Var160', 'Var163',
       'Var173', 'Var181', 'Var189', 'Var192', 'Var193', 'Var194',
       'Var195', 'Var196', 'Var197', 'Var198', 'Var199', 'Var200',
       'Var201', 'Var202', 'Var203', 'Var204', 'Var205', 'Var206',
       'Var207', 'Var208', 'Var210', 'Var211', 'Var212', 'Var214',
       'Var216', 'Var217', 'Var218', 'Var219', 'Var220', 'Var221',
       'Var222', 'Var223', 'Var225', 'Var226', 'Var227', 'Var228',
       'Var229'], dtype=object)

Update numeric and categorical variables, keeping only those that do not exceed the empty entry threshold.

In [58]:
num_vars = [elem for elem in num_vars if elem in keep_vars]
cat_vars = [elem for elem in cat_vars if elem in keep_vars]

In [59]:
print(len(num_vars))
print(len(cat_vars))

42
34


Fill missing values in numeric variables with the column mean and in categorical variables with an additional 'missing' category.

In [60]:
for col in num_vars:
    col_mean = features[col].mean()
    features[col] = features[col].fillna(col_mean)

In [61]:
for col in cat_vars:
    features[col] = features[col].cat.add_categories('missing')
    features[col] = features[col].fillna('missing')

Display the first few rows of the feature data after preprocessing.

In [62]:
features.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1526.000000,7.0,NaN,NaN,NaN,...,oslk,fXVEsaq,jySVZNlOJy,NaN,missing,xb3V,RAYp,F2FyR07IdsN7I,missing,NaN
1,NaN,NaN,NaN,NaN,NaN,525.000000,0.0,NaN,NaN,NaN,...,oslk,2Kb5FSF,LM8l689qOp,NaN,missing,fKCe,RAYp,F2FyR07IdsN7I,missing,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.000000,7.0,NaN,NaN,NaN,...,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN
3,NaN,NaN,NaN,NaN,NaN,1326.437116,0.0,NaN,NaN,NaN,...,oslk,CE7uk3u,LM8l689qOp,NaN,missing,FSa2,RAYp,F2FyR07IdsN7I,missing,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.000000,7.0,NaN,NaN,NaN,...,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN


Calculate the number of unique categories in each categorical variable, keeping only those with less than 50 unique categories.

In [63]:
n_categories_per_feature = features[cat_vars].apply(lambda x: len(set(x)))
n_categories_per_feature

Var192      362
Var193       51
Var194        4
Var195       23
Var196        4
Var197      226
Var198     4291
Var199     5074
Var200    15416
Var201        3
Var202     5714
Var203        6
Var204      100
Var205        4
Var206       22
Var207       14
Var208        3
Var210        6
Var211        2
Var212       81
Var214    15416
Var216     2016
Var217    13991
Var218        3
Var219       23
Var220     4291
Var221        7
Var222     4291
Var223        5
Var225        4
Var226       23
Var227        7
Var228       30
Var229        5
dtype: int64

In [64]:
cat_vars = np.array(n_categories_per_feature[n_categories_per_feature < 5].index)
cat_vars

array(['Var194', 'Var196', 'Var201', 'Var205', 'Var208', 'Var211',
       'Var218', 'Var225'], dtype=object)

In [65]:
len(cat_vars)

8

Finally, select only the kept variables (both numeric and categorical) and display the first few rows of the final feature data.

In [66]:
features = features[list(num_vars) + list(cat_vars)]
features.shape

(50000, 50)

In [67]:
features.head()

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var181,Var189,Var194,Var196,Var201,Var205,Var208,Var211,Var218,Var225
0,1526.000000,7.0,184.0,464.000000,580.0,14.000000,128.0,166.56,0.0,3570.0,...,0.0,462.000000,missing,1K8T,missing,VpdQ,kIsH,L84s,cJvF,missing
1,525.000000,0.0,0.0,168.000000,210.0,2.000000,24.0,353.52,0.0,4764966.0,...,0.0,270.142137,missing,1K8T,missing,sJzTlal,kIsH,L84s,missing,missing
2,5236.000000,7.0,904.0,1212.000000,1515.0,26.000000,816.0,220.08,0.0,5883894.0,...,0.0,270.142137,SEuy,1K8T,smXZ,VpdQ,kIsH,L84s,UYBR,kG3k
3,1326.437116,0.0,0.0,234.518225,0.0,4.507926,0.0,22.08,0.0,0.0,...,0.0,270.142137,missing,1K8T,missing,VpdQ,kIsH,Mtgm,UYBR,missing
4,1029.000000,7.0,3216.0,64.000000,80.0,4.000000,64.0,200.00,0.0,0.0,...,0.0,270.142137,SEuy,1K8T,smXZ,sJzTlal,kIsH,L84s,UYBR,kG3k


In [68]:
output_filename = 'churn_preprocessed_data.csv'
preprocessed_data = pd.concat([features, outcome], axis=1)
preprocessed_data.to_csv('../data/processed/'+ output_filename, index=False)
preprocessed_data

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var189,Var194,Var196,Var201,Var205,Var208,Var211,Var218,Var225,Churn
0,1526.000000,7.000000,184.000000,464.000000,580.000000,14.000000,128.00000,166.560000,0.00000,3.570000e+03,...,462.000000,missing,1K8T,missing,VpdQ,kIsH,L84s,cJvF,missing,-1
1,525.000000,0.000000,0.000000,168.000000,210.000000,2.000000,24.00000,353.520000,0.00000,4.764966e+06,...,270.142137,missing,1K8T,missing,sJzTlal,kIsH,L84s,missing,missing,1
2,5236.000000,7.000000,904.000000,1212.000000,1515.000000,26.000000,816.00000,220.080000,0.00000,5.883894e+06,...,270.142137,SEuy,1K8T,smXZ,VpdQ,kIsH,L84s,UYBR,kG3k,-1
3,1326.437116,0.000000,0.000000,234.518225,0.000000,4.507926,0.00000,22.080000,0.00000,0.000000e+00,...,270.142137,missing,1K8T,missing,VpdQ,kIsH,Mtgm,UYBR,missing,-1
4,1029.000000,7.000000,3216.000000,64.000000,80.000000,4.000000,64.00000,200.000000,0.00000,0.000000e+00,...,270.142137,SEuy,1K8T,smXZ,sJzTlal,kIsH,L84s,UYBR,kG3k,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,357.000000,0.000000,0.000000,132.000000,165.000000,2.000000,0.00000,288.080000,0.00000,6.042420e+06,...,270.142137,missing,1K8T,missing,VpdQ,kIsH,L84s,cJvF,missing,-1
49996,1078.000000,0.000000,2736.000000,380.000000,475.000000,2.000000,88.00000,166.560000,0.00000,0.000000e+00,...,396.000000,SEuy,1K8T,smXZ,09_Q,kIsH,Mtgm,UYBR,kG3k,-1
49997,2807.000000,7.000000,1460.000000,568.000000,710.000000,4.000000,328.00000,166.560000,0.00000,4.221000e+04,...,270.142137,missing,1K8T,missing,VpdQ,kIsH,L84s,UYBR,ELof,-1
49998,1326.437116,6.809496,1249.688401,234.518225,290.245382,4.507926,96.82701,224.507669,0.71681,2.579107e+06,...,276.000000,missing,1K8T,missing,09_Q,kIsH,Mtgm,UYBR,missing,-1


In [69]:
preprocessed_data = pd.get_dummies(preprocessed_data, dtype=int)
preprocessed_data

,Var6,Var7,Var13,Var21,Var22,Var24,Var25,Var28,Var35,Var38,...,Var211_L84s,Var211_Mtgm,Var211_missing,Var218_UYBR,Var218_cJvF,Var218_missing,Var225_ELof,Var225_kG3k,Var225_xG3x,Var225_missing
0,1526.000000,7.000000,184.000000,464.000000,580.000000,14.000000,128.00000,166.560000,0.00000,3.570000e+03,...,1,0,0,0,1,0,0,0,0,1
1,525.000000,0.000000,0.000000,168.000000,210.000000,2.000000,24.00000,353.520000,0.00000,4.764966e+06,...,1,0,0,0,0,1,0,0,0,1
2,5236.000000,7.000000,904.000000,1212.000000,1515.000000,26.000000,816.00000,220.080000,0.00000,5.883894e+06,...,1,0,0,1,0,0,0,1,0,0
3,1326.437116,0.000000,0.000000,234.518225,0.000000,4.507926,0.00000,22.080000,0.00000,0.000000e+00,...,0,1,0,1,0,0,0,0,0,1
4,1029.000000,7.000000,3216.000000,64.000000,80.000000,4.000000,64.00000,200.000000,0.00000,0.000000e+00,...,1,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,357.000000,0.000000,0.000000,132.000000,165.000000,2.000000,0.00000,288.080000,0.00000,6.042420e+06,...,1,0,0,0,1,0,0,0,0,1
49996,1078.000000,0.000000,2736.000000,380.000000,475.000000,2.000000,88.00000,166.560000,0.00000,0.000000e+00,...,0,1,0,1,0,0,0,1,0,0
49997,2807.000000,7.000000,1460.000000,568.000000,710.000000,4.000000,328.00000,166.560000,0.00000,4.221000e+04,...,1,0,0,1,0,0,1,0,0,0
49998,1326.437116,6.809496,1249.688401,234.518225,290.245382,4.507926,96.82701,224.507669,0.71681,2.579107e+06,...,0,1,0,1,0,0,0,0,0,1


In [74]:
preprocessed_data_sample = preprocessed_data.sample(n=1000)

In [75]:
exp = setup(preprocessed_data_sample, target='Churn')
best_model = compare_models(sort='AUC')
best_model

,Description,Value
0,Session id,3425
1,Target,Churn
2,Target type,Binary
3,Target mapping,"-1: 0, 1: 1"
4,Original data shape,"(1000, 72)"
5,Transformed data shape,"(1000, 72)"
6,Transformed train set shape,"(700, 72)"
7,Transformed test set shape,"(300, 72)"
8,Numeric features,71
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:41:14
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Light Gradient Boosting Machine


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.8986,0.6069,0.8986,0.8526,0.8748,-0.0156,-0.0216,0.0260
gbc,Gradient Boosting Classifier,0.9143,0.5899,0.9143,0.8594,0.8838,0.0161,0.0248,0.0710
lda,Linear Discriminant Analysis,0.9000,0.5846,0.9000,0.8553,0.8762,-0.0083,-0.0095,0.0110
ridge,Ridge Classifier,0.9171,0.5599,0.9171,0.8513,0.8829,-0.0085,-0.0098,0.0100
nb,Naive Bayes,0.4414,0.5379,0.4414,0.8547,0.5490,0.0056,0.0043,0.0080
dt,Decision Tree Classifier,0.8371,0.5300,0.8371,0.8653,0.8502,0.0536,0.0535,0.0080
lr,Logistic Regression,0.9229,0.5299,0.9229,0.8517,0.8858,0.0000,0.0000,0.2410
qda,Quadratic Discriminant Analysis,0.7914,0.5287,0.7914,0.8627,0.7871,0.0377,0.0436,0.0090
et,Extra Trees Classifier,0.9186,0.5063,0.9186,0.8514,0.8837,-0.0075,-0.0107,0.0470
rf,Random Forest Classifier,0.9214,0.4939,0.9214,0.8516,0.8851,-0.0024,-0.0033,0.0640


Processing:   0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:
# Split the data into features and target
X = preprocessed_data.drop('Churn', axis=1)
y = preprocessed_data['Churn']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the AdaBoostClassifier model
base_model = AdaBoostClassifier(algorithm='SAMME')

# Define the parameters for tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1, verbose=0)

# Perform model tuning
start_time = time.time()
grid_search.fit(X_train, y_train)
end_time = time.time()

# Best model found
best_model = grid_search.best_estimator_

cv_score = cross_val_score(best_model, X_train, y_train, cv=5, scoring='roc_auc')
print("Cross-validation ROC AUC score:", cv_score.mean())

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate ROC AUC score on the test set
test_roc_auc = roc_auc_score(y_test, y_pred)
print("Test ROC AUC score:", test_roc_auc)

# Save the model
model_name = '../models/best_model.joblib'
dump(best_model, model_name)

# Print the time taken for GridSearchCV
print("GridSearchCV took {:.2f} seconds".format(end_time - start_time))

Test ROC AUC score: 0.5006119553577987
GridSearchCV took 380.83 seconds
